# Simple Text Classification

## Packages & Libraries

In [ ]:
# Sentence Transformers: Multilingual Sentence, Paragraph, and Image Embeddings using BERT & Co.
!pip install sentence-transformers
!pip install git+https://github.com/MartinoMensio/spacy-sentence-bert.git

# for model paraphrase-xlm-r-multilingual-v1
!pip install https://github.com/MartinoMensio/spacy-sentence-bert/releases/download/v0.1.2/xx_paraphrase_xlm_r_multilingual_v1-0.1.2.tar.gz # xx_paraphrase_xlm_r_multilingual_v1-0.1.2

# for model distiluse-base-multilingual-cased-v2
!pip install https://github.com/MartinoMensio/spacy-sentence-bert/releases/download/v0.1.2/xx_distiluse_base_multilingual_cased_v2-0.1.2.tar.gz #xx_distiluse_base_multilingual_cased_v2-0.1.2

# Stem for Indonesian Language
!pip install Sastrawi

In [87]:
import pandas as pd
import numpy as np
from numpy import array


#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# stemming for Indonesian Language
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')
nltk.download('stopwords')


#for model-building
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.cluster import KMeans


#for word embedding
import gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

# for NLP use BERT
# from sentence_transformers import SentenceTransformer
# import spacy_sentence_bert

import warnings
warnings.filterwarnings(action = 'ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Preparation


In [60]:
raw_text = {
    "text" : [
        "Timnas Indonesia menang 2-1 atas Vietnam dalam pertandingan persahabatan.",
        "Pemerintah akan menaikkan harga BBM pada bulan depan.",
        "Pemain basket LeBron James meraih gelar MVP untuk kali ke-4.",
        "Bank sentral menurunkan suku bunga acuan untuk mendorong pertumbuhan ekonomi.",
        "Liverpool memenangkan pertandingan melawan Chelsea dengan skor 2-0.",
        "Saya suka berlari di taman pada pagi hari.",
        "Olahraga sepak bola sangat populer di seluruh dunia.",
        "Lapangan bola basket di taman ini selalu dipenuhi orang.",
        "Bersepeda merupakan olahraga yang baik untuk kesehatan.",
        "Saya sangat senang menjadi bagian dari tim bola voli di sekolah.",
        "Menonton pertandingan sepak bola di stadion adalah pengalaman yang menyenangkan.",
        "Latihan rutin membuat saya merasa lebih sehat dan bugar.",
        "Saya suka berenang di kolam renang umum di akhir pekan.",
        "Mengikuti kelas yoga membantu saya untuk meredakan stres.",
        "Bermain tenis selalu menjadi kegiatan favorit saya di akhir pekan.",
        "Tim renang sekolah memenangkan kejuaraan regional tahun ini.",
        "Bermain futsal bersama teman-teman adalah kegiatan rutin kami setiap minggu.",
        "Saya selalu berolahraga setiap pagi sebelum memulai aktivitas.",
        "Tim atletik sekolah sukses meraih banyak medali pada kompetisi akhir pekan lalu.",
        "Saya menikmati bermain golf bersama ayah di lapangan golf setiap akhir pekan.",
        "Saya suka membaca novel di waktu luang saya.",
        "Film yang saya tonton kemarin malam sangat menyenangkan.",
        "Saya sangat menikmati acara konser musik di stadion besar kemarin.",
        "Hari ini saya mengunjungi pameran seni di museum kota.",
        "Saya selalu merasa tenang saat melakukan meditasi pagi hari.",
        "Menjelajahi alam dan melakukan hiking merupakan kegiatan yang menyenangkan.",
        "Saya suka berlibur di pantai selama akhir pekan.",
        "Menghabiskan waktu dengan keluarga dan teman-teman selalu membuat saya bahagia.",
        "Saya suka belanja di pusat perbelanjaan pada akhir pekan.",
        "Saya sangat menikmati makan siang bersama teman-teman di restoran yang baru dibuka.",
        "Hari ini saya menghadiri seminar tentang pengembangan bisnis.",
        "Saya suka menonton film dokumenter tentang kehidupan hewan.",
        "Saya mengikuti kelas bahasa Inggris untuk meningkatkan kemampuan berbahasa.",
        "Saya sangat senang mengunjungi tempat-tempat bersejarah.",
        "Saya suka membaca buku tentang filsafat di waktu senggang saya.",
        "Saya sering memasak di rumah sebagai hobi.",
        "Saya suka menonton drama Korea pada malam hari.",
        "Saya mengikuti kelas tari modern sebagai hobi.",
        "Saya selalu tertarik dengan teknologi terbaru dan inovatif.",
        "Saya suka bermain game di konsol game kesayangan saya.",
    ],
    "label" : [
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
        "non-olahraga",
    ]
}

In [61]:
df = pd.DataFrame(raw_text)

In [62]:
df

,text,label
0,Timnas Indonesia menang 2-1 atas Vietnam dalam...,olahraga
1,Pemerintah akan menaikkan harga BBM pada bulan...,olahraga
2,Pemain basket LeBron James meraih gelar MVP un...,olahraga
3,Bank sentral menurunkan suku bunga acuan untuk...,olahraga
4,Liverpool memenangkan pertandingan melawan Che...,olahraga
5,Saya suka berlari di taman pada pagi hari.,olahraga
6,Olahraga sepak bola sangat populer di seluruh ...,olahraga
7,Lapangan bola basket di taman ini selalu dipen...,olahraga
8,Bersepeda merupakan olahraga yang baik untuk k...,olahraga
9,Saya sangat senang menjadi bagian dari tim bol...,olahraga


## Data Preprocessing

In [63]:
def preprocess(text):
    """convert to lowercase, strip and remove punctuations"""
    text = text.lower() 

    # remove whitespace
    text = text.strip() 
    # compile pattern that have `<.*?>` then replace with ''
    text = re.compile('<.*?>').sub('', text) 

    # compile pattern that have punctuation !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~ then replace with ' '
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)

    # replace all one or more space to one space
    text = re.sub('\s+', ' ', text)

    # replace all number 0-9 to one space
    text = re.sub(r"\d+", "", text)

    # replace all first word or space in the beginning of line to ''
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    # replace all digits to one space
    text = re.sub(r'\d',' ',text) 

    # replace all one or more space to one space
    text = re.sub(r'\s+',' ',text) 

    return text

def stopword(string):
    """Stopword removal like I, The, An, be, is, etc"""
    a = [i for i in string.split() if i not in stopwords.words('indonesian')]
    return ' '.join(a)

# sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def to_base_word(string):
    """Change words to base words"""
    string = string.split()
    text = [stemmer.stem(word) for word in string]
    return ' '.join(text)

In [64]:
text_data = df["text"]

In [65]:
preprocessed_text = []
for text in text_data:
    preprocessed_text.append(preprocess(str(text)))

preprocessed_text = pd.Series(preprocessed_text)

removed_stopword_text = []
for text in preprocessed_text:
    removed_stopword_text.append(stopword(str(text)))

removed_stopword_text = pd.Series(removed_stopword_text)

cleaned_text = []
for text in removed_stopword_text:
    cleaned_text.append(to_base_word(str(text)))

cleaned_text = pd.Series(cleaned_text)

In [66]:
new_cleaned_df = pd.DataFrame((cleaned_text), columns=["text"])

In [67]:
new_cleaned_df

,text
0,timnas indonesia menang vietnam tanding sahabat
1,perintah naik harga bbm
2,main basket lebron james raih gelar mvp kali
3,bank sentral turun suku bunga acu dorong tumbu...
4,liverpool menang tanding lawan chelsea skor
5,suka lari taman pagi
6,olahraga sepak bola populer dunia
7,lapang bola basket taman penuh orang
8,sepeda olahraga sehat
9,senang tim bola voli sekolah


In [68]:
labels = df["label"]
datas = new_cleaned_df["text"]

In [69]:
new_df = pd.concat([datas, labels], axis=1)

In [70]:
new_df

,text,label
0,timnas indonesia menang vietnam tanding sahabat,olahraga
1,perintah naik harga bbm,olahraga
2,main basket lebron james raih gelar mvp kali,olahraga
3,bank sentral turun suku bunga acu dorong tumbu...,olahraga
4,liverpool menang tanding lawan chelsea skor,olahraga
5,suka lari taman pagi,olahraga
6,olahraga sepak bola populer dunia,olahraga
7,lapang bola basket taman penuh orang,olahraga
8,sepeda olahraga sehat,olahraga
9,senang tim bola voli sekolah,olahraga


## Feature Extraction

In [81]:
vectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english', min_df = 0)

In [88]:
response = vectorizer.fit_transform(new_df["label"])
classes = array(new_df["label"])

## Modelling

In [73]:
from sklearn.model_selection import train_test_split

In [75]:
X_train, X_test, y_train, y_test = train_test_split(new_df['text'], new_df['label'], test_size=0.25, random_state=42)

In [90]:
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(response, classes)
clf.predict(response)

array(['olahraga', 'olahraga', 'olahraga', 'olahraga', 'olahraga',
       'olahraga', 'olahraga', 'olahraga', 'olahraga', 'olahraga',
       'olahraga', 'olahraga', 'olahraga', 'olahraga', 'olahraga',
       'olahraga', 'olahraga', 'olahraga', 'olahraga', 'olahraga',
       'non-olahraga', 'non-olahraga', 'non-olahraga', 'non-olahraga',
       'non-olahraga', 'non-olahraga', 'non-olahraga', 'non-olahraga',
       'non-olahraga', 'non-olahraga', 'non-olahraga', 'non-olahraga',
       'non-olahraga', 'non-olahraga', 'non-olahraga', 'non-olahraga',
       'non-olahraga', 'non-olahraga', 'non-olahraga', 'non-olahraga'],
      dtype=object)

In [91]:
accuracy_score(response, classes)

ValueError: ignored